<a href="https://colab.research.google.com/github/Astroneko404/TF_Study_Note/blob/master/MNIST%20Digit.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from sklearn import datasets
from sklearn.model_selection import train_test_split
import tensorflow as tf

Softmax函数

In [ ]:
def softmax(x):
    import numpy as np
    return np.exp(x) / np.sum(np.exp(x), axis=0)

### Test 1
使用了sklearn的Digit Dataset（8*8）

##### 对Digit Dataset的预处理

In [ ]:
digits = datasets.load_digits()

# Flatten the image to a vector
digits_flattened = []
for img in digits.images:
    digits_flattened.append(img.flatten())
digits_flattened = np.array(digits_flattened)
# digits_flattened = softmax(digits_flattened)

print(len(digits_flattened[0]))

64


In [ ]:
target_one_hot = []
for result in digits.target:
    new_row = [0 for i in range(10)]
    new_row[result] = 1
    target_one_hot.append(new_row)
target_one_hot = np.array(target_one_hot)

print(target_one_hot[:2])

[[1 0 0 0 0 0 0 0 0 0]
 [0 1 0 0 0 0 0 0 0 0]]


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(digits_flattened,
                                                    target_one_hot,
                                                    random_state=0,
                                                    shuffle=False,
                                                    test_size=0.2
                                                    )
print("Train set has size {}".format(len(X_train)))
print("Test set has size {}".format(len(X_test)))
print(X_train[0])
print(y_train[0])

Train set has size 1437
Test set has size 360
[ 0.  0.  5. 13.  9.  1.  0.  0.  0.  0. 13. 15. 10. 15.  5.  0.  0.  3.
 15.  2.  0. 11.  8.  0.  0.  4. 12.  0.  0.  8.  8.  0.  0.  5.  8.  0.
  0.  9.  8.  0.  0.  4. 11.  0.  1. 12.  7.  0.  0.  2. 14.  5. 10. 12.
  0.  0.  0.  0.  6. 13. 10.  0.  0.  0.]
[1 0 0 0 0 0 0 0 0 0]


##### Tensorflow模型
代码来自[TensorFlow入门(一) - mnist手写数字识别(网络搭建)](https://geektutu.com/post/tensorflow-mnist-simplest.html)

In [ ]:
class Network:
    def __init__(self):
        # 学习速率(一般在 0.00001 - 0.5 之间)
        self.learning_rate = 0.01

        # 图片一维向量
        self.x = tf.compat.v1.placeholder(tf.float32, [None, 64])

        # 标签值，即图像对应的结果，如果对应数字是8，则对应label是 [0,0,0,0,0,0,0,0,1,0]
        # 这种方式称为 one-hot编码
        # 标签是一个长度为10的一维向量，值最大的下标即图片上写的数字
        self.label = tf.compat.v1.placeholder(tf.float32, [None, 10])

        # 权重，初始化全 0
        self.w = tf.Variable(tf.zeros([64, 10]))
        # 偏置 bias，初始化全 0
        self.b = tf.Variable(tf.zeros([10]))
        # 输出 y = softmax(X * w + b)
        self.y = tf.nn.softmax(tf.matmul(self.x, self.w) + self.b)

        # 损失，即交叉熵，最常用的计算标签(label)与输出(y)之间差别的方法
        self.loss = -tf.reduce_sum(input_tensor=self.label * tf.math.log(self.y + 1e-10))

        # 反向传播，采用梯度下降的方法。调整w与b，使得损失(loss)最小
        # loss越小，那么计算出来的y值与 标签(label)值越接近，准确率越高
        self.train = tf.compat.v1.train.GradientDescentOptimizer(self.learning_rate).minimize(self.loss)

        # 以下代码验证正确率时使用
        # argmax 返回最大值的下标，最大值的下标即答案
        # 例如 [0,0,0,0.9,0,0.1,0,0,0,0] 代表数字3
        predict = tf.equal(tf.argmax(input=self.label, axis=1), tf.argmax(input=self.y, axis=1))

        # predict -> [true, true, true, false, false, true]
        # reduce_mean即求predict的平均数 即 正确个数 / 总数，即正确率
        self.accuracy = tf.reduce_mean(input_tensor=tf.cast(predict, "float"))

In [ ]:
class Train:
    def __init__(self):
        self.net = Network()

        # 初始化 session
        # Network() 只是构造了一张计算图，计算需要放到会话(session)中
        self.sess = tf.compat.v1.Session()
        # 初始化变量
        self.sess.run(tf.compat.v1.global_variables_initializer())

    def train(self):
        # batch_size 是指每次迭代训练，传入训练的图片张数。
        # 数据集小，可以使用全数据集，数据大的情况下，
        # 为了提高训练速度，用随机抽取的n张图片来训练，效果与全数据集相近
        # https://www.zhihu.com/question/32673260
        batch_size = 70

        # 总的训练次数
        train_step = 20

        # 开始训练
        for i in range(train_step):
            # 从数据集中获取 输入和标签(也就是答案)
            x, label = X_train[i*batch_size:(i+1)*batch_size-1], y_train[i*batch_size:(i+1)*batch_size-1]

            # 每次计算train，更新整个网络
            # loss只是为了看到损失的大小，方便打印
            _, loss = self.sess.run([self.net.train, self.net.loss],
                                    feed_dict={self.net.x: x, self.net.label: label})

            # 打印 loss，训练过程中将会看到，loss有变小的趋势
            # 代表随着训练的进行，网络识别图像的能力提高
            # 但是由于网络规模较小，后期没有明显下降，而是有明显波动
            if (i + 1) % 10 == 0:
                print('第%5d步，当前loss：%.2f' % (i + 1, loss))
    
    def calculate_accuracy(self):
        test_x = X_test
        test_label = y_test

        # 注意：与训练不同的是，并没有计算 self.net.train
        # 只计算了accuracy这个张量，所以不会更新网络
        # 最终准确率约为0.91
        accuracy = self.sess.run(self.net.accuracy,
                                 feed_dict={self.net.x: test_x, self.net.label: test_label})
        print("准确率: %.2f，共测试了%d张图片 " % (accuracy, len(test_label)))

if __name__ == "__main__":
    tf.compat.v1.disable_eager_execution()
    app = Train()
    app.train()
    app.calculate_accuracy()

第   10步，当前loss：1174.32
第   20步，当前loss：1036.16
准确率: 0.28，共测试了360张图片 


### Test 2
使用了MNIST数据集（原文版本）

##### MNIST Dataset预处理

In [ ]:
data = tf.keras.datasets.mnist.load_data()
X_train, y_train, X_test, y_test = data[0][0], data[0][1], data[1][0], data[1][1]

# Flatten the image to a vector
X_train_flattened = []
for img in X_train:
    X_train_flattened.append(img.flatten())
X_train_flattened = np.array(X_train_flattened) / 255.0

X_test_flattened = []
for img in X_test:
    X_test_flattened.append(img.flatten())
X_test_flattened = np.array(X_test_flattened) / 255.0

# One-hot transform
y_train_one_hot = []
for result in y_train:
    new_row = [0 for i in range(10)]
    new_row[result] = 1
    y_train_one_hot.append(new_row)
y_train_one_hot = np.array(y_train_one_hot)

y_test_one_hot = []
for result in y_test:
    new_row = [0 for i in range(10)]
    new_row[result] = 1
    y_test_one_hot.append(new_row)
y_test_one_hot = np.array(y_test_one_hot)

print("X_train:", X_train_flattened.shape)
print("y_train:", y_train_one_hot.shape)
print("X_test:", X_test_flattened.shape)
print("y_test:", y_test_one_hot.shape)

X_train: (60000, 784)
y_train: (60000, 10)
X_test: (10000, 784)
y_test: (10000, 10)


##### 模型

In [ ]:
class Network:
    def __init__(self):
        # 学习速率(一般在 0.00001 - 0.5 之间)
        self.learning_rate = 0.0001

        # 图片一维向量
        self.x = tf.compat.v1.placeholder(tf.float32, [None, 784])

        # 标签值，即图像对应的结果，如果对应数字是8，则对应label是 [0,0,0,0,0,0,0,0,1,0]
        # 这种方式称为 one-hot编码
        # 标签是一个长度为10的一维向量，值最大的下标即图片上写的数字
        self.label = tf.compat.v1.placeholder(tf.float32, [None, 10])

        # 权重，初始化全 0
        self.w = tf.Variable(tf.zeros([784, 10]))
        # 偏置 bias，初始化全 0
        self.b = tf.Variable(tf.zeros([10]))
        # 输出 y = softmax(X * w + b)
        self.y = tf.nn.softmax(tf.matmul(self.x, self.w) + self.b)

        # 损失，即交叉熵，最常用的计算标签(label)与输出(y)之间差别的方法
        self.loss = -tf.reduce_sum(input_tensor=self.label * tf.math.log(self.y + 1e-10))

        # 反向传播，采用梯度下降的方法。调整w与b，使得损失(loss)最小
        # loss越小，那么计算出来的y值与 标签(label)值越接近，准确率越高
        self.train = tf.compat.v1.train.GradientDescentOptimizer(self.learning_rate).minimize(self.loss)

        # 以下代码验证正确率时使用
        # argmax 返回最大值的下标，最大值的下标即答案
        # 例如 [0,0,0,0.9,0,0.1,0,0,0,0] 代表数字3
        predict = tf.equal(tf.argmax(input=self.label, axis=1), tf.argmax(input=self.y, axis=1))

        # predict -> [true, true, true, false, false, true]
        # reduce_mean即求predict的平均数 即 正确个数 / 总数，即正确率
        self.accuracy = tf.reduce_mean(input_tensor=tf.cast(predict, "float"))

In [ ]:
# from tensorflow.examples.tutorials.mnist import input_data

class Train:
    def __init__(self):
        self.net = Network()

        # 初始化 session
        # Network() 只是构造了一张计算图，计算需要放到会话(session)中
        self.sess = tf.compat.v1.Session()
        # 初始化变量
        self.sess.run(tf.compat.v1.global_variables_initializer())

    def train(self):
        # batch_size 是指每次迭代训练，传入训练的图片张数。
        # 数据集小，可以使用全数据集，数据大的情况下，
        # 为了提高训练速度，用随机抽取的n张图片来训练，效果与全数据集相近
        # https://www.zhihu.com/question/32673260
        batch_size = 5120

        # 总的训练次数
        train_step = 1000

        # 开始训练
        for i in range(train_step):
            x, label = X_train_flattened[i*batch_size:(i+1)*batch_size-1],\
                       y_train_one_hot[i*batch_size:(i+1)*batch_size-1]

            # 每次计算train，更新整个网络
            # loss只是为了看到损失的大小，方便打印
            _, loss = self.sess.run([self.net.train, self.net.loss],
                                    feed_dict={self.net.x: x, self.net.label: label})

            # 打印 loss，训练过程中将会看到，loss有变小的趋势
            # 代表随着训练的进行，网络识别图像的能力提高
            # 但是由于网络规模较小，后期没有明显下降，而是有明显波动
            if (i + 1) % 500 == 0:
                print('第%5d步，当前loss：%.2f' % (i + 1, loss))
    
    def calculate_accuracy(self):
        test_x = X_test_flattened
        test_label = y_test_one_hot

        accuracy = self.sess.run(self.net.accuracy,
                                 feed_dict={self.net.x: test_x, self.net.label: test_label})
        print("准确率: %.2f，共测试了%d张图片 " % (accuracy, len(test_label)))

if __name__ == "__main__":
    print("Tensorflow version:", tf.__version__)
    tf.compat.v1.disable_eager_execution()
    app = Train()
    app.train()
    app.calculate_accuracy()

Tensorflow version: 2.3.0
第  500步，当前loss：-0.00
第 1000步，当前loss：-0.00
准确率: 0.86，共测试了10000张图片 


### Test 3
LSTM模型+MNIST数据集

##### 数据处理

In [ ]:
data = tf.keras.datasets.mnist.load_data()
X_train, y_train, X_test, y_test = data[0][0], data[0][1], data[1][0], data[1][1]

X_train, X_test = X_train / 255.0, X_test / 255.0

# y_train_one_hot = []
# for result in y_train:
#     new_row = [0 for i in range(10)]
#     new_row[result] = 1
#     y_train_one_hot.append(new_row)
# y_train_one_hot = np.array(y_train_one_hot)

# y_test_one_hot = []
# for result in y_test:
#     new_row = [0 for i in range(10)]
#     new_row[result] = 1
#     y_test_one_hot.append(new_row)
# y_test_one_hot = np.array(y_test_one_hot)

print("X_train:", X_train.shape)
print("y_train:", y_train.shape)
print("X_test:", X_test.shape)
print("y_test:", y_test.shape)

[5 0 4 ... 5 6 8]
X_train: (60000, 28, 28)
y_train: (60000, 10)
X_test: (10000, 28, 28)
y_test: (10000, 10)


##### 模型

In [ ]:
from tensorflow.keras import models, layers, optimizers

inputs = layers.Input(shape=(28, 28), name='inputs')
lstm = layers.LSTM(units=128, return_sequences=False)(inputs)
outputs = layers.Dense(10, activation='softmax')(lstm)
lstm = tf.keras.Model(inputs, outputs)

In [ ]:
lstm.compile(optimizer=tf.keras.optimizers.Adam(0.001),
             loss='sparse_categorical_crossentropy',
             metrics=['accuracy'])
lstm.summary()

Model: "functional_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
inputs (InputLayer)          [(None, 28, 28)]          0         
_________________________________________________________________
lstm_3 (LSTM)                (None, 128)               80384     
_________________________________________________________________
dense_3 (Dense)              (None, 10)                1290      
Total params: 81,674
Trainable params: 81,674
Non-trainable params: 0
_________________________________________________________________


In [ ]:
lstm.fit(X_train, y_train, batch_size=32, epochs=5, validation_split=0.1)
lstm.evaluate(X_test,  y_test, verbose=2)

Train on 54000 samples, validate on 6000 samples
Epoch 1/5
54000/54000 [==============================] - 48s 887us/sample - loss: 0.0475 - accuracy: 0.9851 - val_loss: 0.0478 - val_accuracy: 0.9872
Epoch 2/5
54000/54000 [==============================] - 49s 909us/sample - loss: 0.0395 - accuracy: 0.9880 - val_loss: 0.0490 - val_accuracy: 0.9872
Epoch 3/5
54000/54000 [==============================] - 49s 906us/sample - loss: 0.0329 - accuracy: 0.9900 - val_loss: 0.0483 - val_accuracy: 0.9878
Epoch 4/5
54000/54000 [==============================] - 49s 898us/sample - loss: 0.0294 - accuracy: 0.9907 - val_loss: 0.0413 - val_accuracy: 0.9880
Epoch 5/5
54000/54000 [==============================] - 49s 902us/sample - loss: 0.0249 - accuracy: 0.9921 - val_loss: 0.0439 - val_accuracy: 0.9882


[0.05914884710437618, 0.9813]